In [11]:
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('./data/tmp.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Vaccine'].unique(),
                'Covaxin',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Vaccine'].unique(),
                'Covishield',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'India'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))

def update_graph(
    xaxis_column_name='Oxford/AstraZeneca',
    yaxis_column_name='Covishield',
    xaxis_type='Linear',
    yaxis_type='Linear',
    year_value=2024
):
  dff = df[df['Year'] == year_value]

  # Extract single column values for x and y axes
  x_values = dff[dff['Vaccine'] == xaxis_column_name]['Vaccinated']
  y_values = dff[dff['Vaccine'] == yaxis_column_name]['Vaccinated']

  # Check if filtered DataFrames are empty
  if x_values.empty or y_values.empty:
      # Handle empty data case
      return {  # Return a dictionary for the figure (Dash requirement)
          'data': [],  # Empty list for data traces
          'layout': {  # Basic layout with title
              'title': 'No data available for this selection'
          }
      }

  # Data is available, proceed with figure creation
  fig = px.scatter(x=x_values,
                   y=y_values,
                   hover_name=dff[dff['Vaccine'] == yaxis_column_name]['Country Name'],
                   title='Vaccine Comparison - Year {}'.format(year_value),  # Set title in px.scatter
                   )

  fig.update_traces(customdata=dff[dff['Vaccine'] == yaxis_column_name]['Country Name'])

  fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

  fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

  fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

  return fig



def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Vaccinated')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Vaccine'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Vaccine'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run(debug=True,port=8092)


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[11], line 101, in update_graph(
    xaxis_column_name='Novavax',
    yaxis_column_name='Covishield',
    xaxis_type='Linear',
    yaxis_type='Linear',
    year_value=2024
)
     93     return {  # Return a dictionary for the figure (Dash requirement)
     94         'data': [],  # Empty list for data traces
     95         'layout': {  # Basic layout with title
     96             'title': 'No data available for this selection'
     97         }
     98     }
    100 # Data is available, proceed with figure creation
--> 101 fig = px.scatter(x=x_values,
        x_values = 161      568768.0
246    30726906.0
Name: Vaccinated, dtype: float64
        px = <module 'plotly.express' from 'c:\\Users\\admin\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\plotly\\express\\__init__.py'>
        y_values = 10    250000.0
N